# Лабораторная работа №1

In [5]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет со свойствами различных грибов, целевым параметром является съедобность (значение "e" целевого параметра) или несъедобность ("p") гриба.

In [6]:
mushrooom_dataset = pd.read_csv('MushroomDataset/secondary_data.csv',delimiter=';')

Выделяем таргетный столбец в отдельный Series, а все остальные столбы в DataFrame. Также удаляем все столбцы, где большая часть данных типа NaN и заменяем все классы на номер буквы в алфавите.

In [7]:
y = mushrooom_dataset['class']
X = mushrooom_dataset.drop('class',axis=1)
X = X.drop(columns=['cap-surface','gill-spacing','gill-attachment','stem-root',
                    'stem-surface','veil-type','veil-color','ring-type','spore-print-color'])
X.replace({'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,
           'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,
           'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,
           't':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26},inplace=True)

После устранения пропусков нормализуем все значения.

In [8]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем данные на тренировочную и тестовую части.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.2)

Далее найдём лучшие гиперпараметры, для этого передадим в GridSearchCV тренировочные данные, и проведём тренировку для кол-ва соседей от 1 до 30, и 3-х видов метрик, после чего запомним самые лучшие параметры.

In [10]:
kn_classifier = KNeighborsClassifier()
kn_params ={
    'n_neighbors': range(1,30),
    'metric':['minkowski','euclidean','manhattan']  
}
kn_grid = GridSearchCV(kn_classifier, kn_params, cv=5, n_jobs=-1)
kn_grid.fit(X_train, y_train)
kn_best_params = kn_grid.best_params_

После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [11]:
kn_best_model = KNeighborsClassifier(**kn_best_params)
kn_best_model.fit(X_train, y_train)
kn_predicted = kn_best_model.predict(X_test)
print('Used params:',kn_best_params)
print('Evaluation:\n', metrics.classification_report(y_test, kn_predicted,digits=5))

Used params: {'metric': 'manhattan', 'n_neighbors': 1}
Evaluation:
               precision    recall  f1-score   support

           e    0.99520   0.99391   0.99455      5421
           p    0.99515   0.99617   0.99566      6793

    accuracy                        0.99517     12214
   macro avg    0.99517   0.99504   0.99511     12214
weighted avg    0.99517   0.99517   0.99517     12214

